# Monte Carlo Learning

The key idea of Monte Carlo methods is to replace the explicit transition structure used by other by approximating it from average returns run over a number of simulated episodes. Here we assume that the experience gathered by the agent is dividen into episodes that eventually finish. 

We will look at **episodic** problems first: this means that there is a time where the game/interaction terminates, for instance, if a final state is reached. 

Monte Carlo Learning is model-free because we don't need to know the rewards or transitions of the underlying MDP. In particular, the estimates for one state do not "build upon" estimates of the other. This is useful if you only require the value at certain states. The main drawback is that it might take too much time, even for small problems. 



## Monte Carlo Prediction

Let's distinguish between **prediction** and **control** parts of our RL problem. The prediction part tells us, for a *fixed* policy, how much is that policy worth, whereas the control part tells us how to update our policy to obtain an optimal (or close to optimal).

The goal is to learn $Q_\pi$ from episodes of experience under $\pi$:
$$S_1, A_1, R_1, S_2, A_2, R_2, \ldots S_T \sim \pi$$
Recall that the return $G_t$ is
$$G_t = R_{t+1}+\gamma R_{t+2}+\ldots + \gamma^{T-1}R_T$$
and that the value function is the expected return
$$Q_\pi(s,a) = \mathbb E_\pi(G_t  \ | \ S_t = s, A_t = a)$$
For Monte Carlo simulation we replace the expectation above by empirical mean.

To ensure that sampled average returns would converge to the value function, we need to verify that:
- All episodes must start in a state-action pair. 
- All state-action pairs have positive probability of being selected at the start.

This guarantees that for a large number of episodes, all pairs would be selected sufficently many times.

<div class="alert alert-block alert-info">
<h4>First-Visit Monte Carlo Policy Evaluation </h4>
<p>We want to evaluate a state $s$ under a fixed policy $\pi$: 
    <ul>
     <li> Increment a counter the first that the pair $s,a$ is visited in an episode
        $$N(s,a) \leftarrow N(s,a)+1.$$ </li>
    <li> Increment total return $R(s,a)\leftarrow R(s,a)+G_t$    </li>
    <li> Let $Q_\pi(s,a) \sim R(s,a)/N(s,a)$ </li>
    <li> $Q_\pi(s,a) \leftarrow Q_\pi(s,a)$ as $N(s,a)\rightarrow +\infty$ </li>    
    </ul>
</div>    

Note that the counter $N(s,a)$ persist across many episodes. At this point, we only want to see how good the policy $\pi$ is, not how to improve it.

A variation of this idea is the **Every-Visit Monte Carlo Policy Evaluation**:


<div class="alert alert-block alert-info">
<h4>Every-Visit Monte Carlo Policy Evaluation </h4>
<p>We want to evaluate a state $s$ under a fixed policy $\pi$: 
    <ul>
     <li> Increment a counter every time that the pair $s,a$ is visited in an episode
        $$N(s,a) \leftarrow N(s,a)+1.$$ </li>
    <li> Increment total return $R(s,a)\leftarrow R(s,a)+G_t$    </li>
    <li> Let $Q_\pi(s,a) \sim R(s,a)/N(s,a)$ </li>
    <li> $Q_\pi(s,a) \leftarrow Q_\pi(s,a)$ as $N(s,a)\rightarrow +\infty$ </li>    
    </ul>
</div>    

What's the difference between these two? Almost none in practice, although the theoretical analysis is different: for instance, notice that the counter $N(s,a)$ is increased several times per episode. Both methods are valid estimators. They are **on-policy**, meaning that they sample and evaluate from the same policy.

Observe that the mean of a sequence $x_1, x_2, \ldots$ can be computed incrementally:

$$\begin{aligned}
\mu_k & = & \frac{1}{k}\sum_{j=1}^k x_j \\
 & = & \frac{1}{k}\left( x_k + \sum_{j=1}^{k-1}x_j\right) \\
 & = & \frac{1}{k}\left( x_k + (k-1)\mu_{k-1}\right) \\
 & = & \mu_{k-1} +\frac{1}{k}(x_k-\mu_{k-1})
\end{aligned}$$



This means we can do incremental Monte Carlo Updates for the $Q$ function, without having to remember the full history:



<div class="alert alert-block alert-info">
<h4>Incremental Monte Carlo Algorithm</h4>
<ul>
     <li>For each state $S_t$ with return $G_t$ </li>
 $$\begin{aligned}
 N(S_t,A_t) &\leftarrow& N(S_t, A_t) +1\\
 Q(S_t,A_t) &\leftarrow& Q(S_t,A_t)+\frac{1}{N(S_t, A_t)}(G_t-Q(S_t,A_t))
 \end{aligned}
 $$
 </ul>
</div> 


We can "forget the past" by compute an exponential moving mean. We don't move to correct the value all the way to the mean, we just correct it a bit.
$$Q(S_t,A_t) \leftarrow Q(S_t,A_t)+\alpha(G_t-Q(S_t,A_t))$$

What's the advantage of this? By fixing the step size, we do not move all the way to the exact estimate of the mean: we do not want to remember our previous estimates of the mean payoff from far in the past.

## Monte Carlo Control

We know how to evaluate policies. How can we do that? One idea would be to do **policy iteration**, after all, we have the value function, so we can do greedy policy iteration, as in the model based case, right?

Well, there's a problem with that: following a greedy policy might not ensure that we will explore the full state-action space! So our estimates via Monte Carlo may not be correct. 

Instead of acting greedily, we can act rather **$\epsilon$-greedy**: with a small probability $\epsilon$, choose a random action. This ensures that we will visit all possible states with positive probability.

<div class="alert alert-block alert-info">
<h4> GLIE Monte Carlo Control </h4>
<p>Sample the $k$-th episode according to the policy $\pi$: 
    <ul>
      <p><b> (Policy Evaluation) </b></p>
     <li> Increment a counter every time that the pair $s,a$ is visited in an episode
        $$N(s,a) \leftarrow N(s,a)+1.$$ </li>
    <li> Increment total return $R(s,a)\leftarrow R(s,a)+G_t$    </li>
    <li> Let $Q(s,a) \sim R(s,a)/N(s,a)$ </li>
    <li> $Q(s,a) \leftarrow Q_\pi(s,a)$ as $N(s,a)\rightarrow +\infty$ </li>    
    <p><b> (Policy Improvement)</b> </p>
    <li> $\epsilon \leftarrow 1/k$ </li>
    </ul>
</div>  


Note that GLIE is an on-policy algorithm: we are evaluating the policy we are learning from. 

For this particular version of GLIE, it does not matter how is $Q$ initialized. 

For the non-stationary version $\left( Q(S_t,A_t) \leftarrow Q(S_t,A_t)+\alpha(G_t-Q(S_t,A_t) \right)$ the initialization of $Q$ might matter more.

For the implementation, we need first to implement an $\epsilon-$greedy policy:

In [2]:
## The code below shows how to implement GLIE.
import numpy as np
import gym

def epsilon_greedy_policy(Q, epsilon, actions):
    """ Q is a numpy array, epsilon between 0,1 
    and a list of actions"""
    
    def policy_fn(state):
        if np.random.rand()>epsilon:
            action = np.argmax(Q[state,:])
        else:
            action = np.random.choice(actions)
        return action
    return policy_fn




Now we need a function that runs the episode:

In [9]:
env = gym.make("FrozenLake-v0")

Q = np.zeros([env.observation_space.n, env.action_space.n])
R = np.zeros([env.observation_space.n, env.action_space.n])
N = np.zeros([env.observation_space.n, env.action_space.n])
actions = range(env.action_space.n)
gamma = 0.99


def run_episode(env, policy): 
    done = False
    state = env.reset()
    episode = []
    while not done:
        action = policy(state)
        new_state, reward, done, _ = env.step(action)
        episode.append((state,action,reward))
        state = new_state    
    return episode

Once this is all in place, we need to set up the training schedule: how are parameters (in this case, the $Q$ function only) going to be updated?

In [10]:
from tqdm import tqdm
score = 0
n_iter = 50000
for j in tqdm(range(n_iter)):
    policy = epsilon_greedy_policy(Q,epsilon=100/(j+1), actions = actions )
    episode = run_episode(env,policy)
    ep_reward = sum(x[2]*(gamma**i) for i, x in enumerate(episode))
    score += ep_reward # counter for the 100 episode reward
    
    sa_in_episode = set([(x[0],x[1]) for x in episode])
    
    # Find first visit of each s,a in the episode
    for s,a in sa_in_episode:
        first_visit = next(i for i,x in enumerate(episode) \
                           if x[0]==s and x[1]==a)
        G = sum(x[2]*(gamma**i) for i, x in enumerate(episode[first_visit:]))
        R[s,a] += G
        N[s,a] += 1
        Q[s,a] = R[s,a]/N[s,a]

    if (j+1)%10000 == 0: print("Score: ", score/100, end="")
    
    if j%100 == 0:
        score = 0
    
env.close()

 20%|██████████████▉                                                            | 9926/50000 [00:06<00:30, 1301.83it/s]

Score:  0.3845999279193392

 40%|█████████████████████████████▍                                            | 19895/50000 [00:14<00:24, 1251.09it/s]

Score:  0.36370057660232397

 60%|████████████████████████████████████████████▏                             | 29885/50000 [00:23<00:15, 1296.51it/s]

...

 80%|███████████████████████████████████████████████████████████               | 39902/50000 [00:31<00:08, 1173.84it/s]

Score:  0.3221021893061096

100%|█████████████████████████████████████████████████████████████████████████▉| 49964/50000 [00:40<00:00, 1333.60it/s]

Score:  0.3700988195029871

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:40<00:00, 1223.82it/s]
